In [25]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
win_size = (48, 96)
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
num_bins = 9
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

In [ ]:
import numpy as np
import random
random.seed(42)

import os
hroi = 128
wroi = 64

X_pos = []
dirs = ['cats/', 'dogs/']
for dir in dirs:
    posdir =  'dataset_dogs_vs_cats/train/' + dir
    for posfile in os.listdir(posdir):
        filename = posdir + posfile
        img = cv2.imread(filename)
        img = cv2.resize(img, (512, 512))
        for j in range(5):
            rand_y = random.randint(0, img.shape[0] - hroi)
            rand_x = random.randint(0, img.shape[1] - wroi)
            roi = img[rand_y:rand_y + hroi, rand_x:rand_x + wroi, :]
            X_pos.append(hog.compute(roi, (64, 64)))

In [ ]:
X_pos = np.array(X_pos, dtype=np.float32)
y_pos = np.ones(X_pos.shape[0], dtype=np.int32)
X_pos.shape, y_pos.shape

In [ ]:
X_neg = []
negdir = 'data/chapter6/pedestrians_neg/'
for negfile in os.listdir(negdir):
    filename = negdir + negfile
    img = cv2.imread(filename)
    img = cv2.resize(img, (512, 512))
    for j in range(5):
        rand_y = random.randint(0, img.shape[0] - hroi)
        rand_x = random.randint(0, img.shape[1] - wroi)
        roi = img[rand_y:rand_y + hroi, rand_x:rand_x + wroi, :]
        X_neg.append(hog.compute(roi, (64, 64)))

In [ ]:
X_neg = np.array(X_neg, dtype=np.float32)
y_neg = -np.ones(X_neg.shape[0], dtype=np.int32)
X_neg.shape, y_neg.shape

In [ ]:
X = np.concatenate((X_pos, X_neg))
y = np.concatenate((y_pos, y_neg))

In [ ]:
from sklearn import model_selection as ms
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
def train_svm(X_train, y_train):
    svm = cv2.ml.SVM_create()
    svm.train(X_train, cv2.ml.ROW_SAMPLE, y_train)
    return svm

In [ ]:
def score_svm(svm, X, y):
    from sklearn import metrics
    _, y_pred = svm.predict(X)
    return metrics.accuracy_score(y, y_pred)

In [ ]:
svm = train_svm(X_train, y_train)

In [ ]:
score_svm(svm, X_train, y_train)

In [ ]:
score_svm(svm, X_test, y_test)

In [ ]:
img_test = cv2.imread('example.jpg')

In [ ]:
stride = 16
found = []
for ystart in np.arange(0, img_test.shape[0], stride):
    for xstart in np.arange(0, img_test.shape[1], stride):
        if ystart + hroi > img_test.shape[0]:
            continue
        if xstart + wroi > img_test.shape[1]:
            continue
        roi = img_test[ystart:ystart + hroi, xstart:xstart + wroi, :]
        feat = np.array([hog.compute(roi, (64, 64))])
        _, ypred = svm.predict(feat)
        if np.allclose(ypred, 1):
            found.append((ystart, xstart, hroi, wroi))

In [ ]:
hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

In [ ]:
rho, _, _ = svm.getDecisionFunction(0)
sv = svm.getSupportVectors()
hog.setSVMDetector(np.append(sv[0, :].ravel(), rho))

In [ ]:
found, _ = hog.detectMultiScale(img_test)

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.imshow(cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB))
from matplotlib import patches
for f in found:
    ax.add_patch(patches.Rectangle((f[0], f[1]), f[2], f[3], color='g', linewidth=5, fill=False))
plt.savefig('detected.png')